# Data Cleaning Notebook

## Dataset Source
This dataset was derived from [Kaggle](https://www.kaggle.com/datasets/ericgoldberg/safety-and-crime-911-calls-new-orleans-2018).

The original dataset can be found [here](https://catalog.data.gov/dataset/calls-for-service-2018).

## Goals of this Project
The goal of this project is to create an analysis and visualization of incident data reported to the Orleans Parish Communication District (OPCD) in the year 2018. Resulting insights could assist NOPD and relevant stakeholders in making informed decisions about resource allocation to improve public safety, reduce crime, and enhance law enforcement effectiveness. Additionally, this analysis could promote transparency and accountability in resource allocation practices.

### Goals of this Notebook
1. Break down coordinate values into usable Latitude and Longitude values for heatmap purposes.
2. Group events into appropriate incident/crime categories to visualize geographic and temporal patterns.
3. Remove burglar alarm-related records, as they are too unreliable for the purposes of this project. 

In [1]:
import pandas as pd
df = pd.read_csv('nolacrime.csv')

In [2]:
df.head()

,Code,Type,Time,Beat,Address,Zip,District,Coordinates,Neighborhood
0,107,SUSPICIOUS PERSON,2018-02-08 16:55:15,2K03,018XX Cambronne St,70118.0,2,"[29.9543012, -90.12741579]",LEONIDAS
1,107,SUSPICIOUS PERSON,2018-02-08 17:20:45,5I04,Andry St & N Claiborne Ave,70117.0,5,"[29.9668437, -90.0168289]",LOWER NINTH WARD
2,62B,BUSINESS BURGLARY,2018-02-08 05:19:41,7L01,046XX Michoud Blvd,70129.0,7,"[30.03203358, -89.92870826]",VILLAGE DE LEST
3,29,DEATH,2018-02-08 10:16:26,6P01,014XX General Taylor St,70115.0,6,"[29.92592877, -90.09646117]",TOURO
4,67,THEFT,2018-02-08 10:54:10,5L01,023XX N Tonti St,70117.0,5,"[29.98070722, -90.05459674]",ST. ROCH


#### 1. Split "Coordinates" column values into new longitude and latitude columns for Tableau heatmap usability. 

1a. Remove brackets and commas from "Coordinates" values, and create new "Latitude" and "Longitude" columns.

In [3]:
df = df.Coordinates.str.strip("[]").str.split(", ")

In [4]:
df = df.apply(pd.Series).astype(float).rename(columns={0: "Latitude", 1: "Longitude"})

In [5]:
df.head()

,Latitude,Longitude
0,29.954301,-90.127416
1,29.966844,-90.016829
2,30.032034,-89.928708
3,29.925929,-90.096461
4,29.980707,-90.054597


In [6]:
df_main = pd.read_csv('nolacrime.csv')

In [7]:
df_main.drop(columns=['Coordinates'], inplace=True)

1b. Remove original "Coordinates" column and merge new Latitude/Longitude dataframe columns onto original dataframe. 

In [8]:
result = pd.merge(df_main, df, left_index=True, right_index=True)

In [9]:
result.head()

,Code,Type,Time,Beat,Address,Zip,District,Neighborhood,Latitude,Longitude
0,107,SUSPICIOUS PERSON,2018-02-08 16:55:15,2K03,018XX Cambronne St,70118.0,2,LEONIDAS,29.954301,-90.127416
1,107,SUSPICIOUS PERSON,2018-02-08 17:20:45,5I04,Andry St & N Claiborne Ave,70117.0,5,LOWER NINTH WARD,29.966844,-90.016829
2,62B,BUSINESS BURGLARY,2018-02-08 05:19:41,7L01,046XX Michoud Blvd,70129.0,7,VILLAGE DE LEST,30.032034,-89.928708
3,29,DEATH,2018-02-08 10:16:26,6P01,014XX General Taylor St,70115.0,6,TOURO,29.925929,-90.096461
4,67,THEFT,2018-02-08 10:54:10,5L01,023XX N Tonti St,70117.0,5,ST. ROCH,29.980707,-90.054597


#### 2. Group incident types into associated categories for better overview visualization. 

2a. Assess unique values for incident "Type" column. 

In [10]:
result.Type.unique()

array(['SUSPICIOUS PERSON', 'BUSINESS BURGLARY', 'DEATH', 'THEFT',
       'BURGLAR ALARM, SILENT', 'PROWLER', 'DOMESTIC DISTURBANCE',
       'FIGHT', 'DISCHARGING FIREARM', 'SIMPLE BATTERY DOMESTIC',
       'PICKPOCKET', 'AUTO THEFT', 'HIT & RUN', 'SHOPLIFTING',
       'NOISE COMPLAINT', 'AGGRAVATED ASSAULT', 'RECKLESS DRIVING',
       'SIMPLE BATTERY', 'DRUG VIOLATIONS',
       'RECOVERY OF REPORTED STOLEN VEHICLE', 'SIMPLE CRIMINAL DAMAGE',
       'FIREWORKS', 'ARMED ROBBERY WITH GUN', 'SIMPLE BURGLARY VEHICLE',
       'UNAUTHORIZED USE OF VEHICLE', 'HOMELESS', 'HOLD UP ALARM', 'FIRE',
       'SIMULTANEOS STOLEN/RECOVERY VEHICLE', 'FUGITIVE ATTACHMENT',
       'BURGLAR ALARM, LOCAL', 'TRUANT VIOLATION', 'MISSING ADULT',
       'QUALITY OF LIFE ISSUE', 'THEFT FROM EXTERIOR', 'SIMPLE BURGLARY',
       'ISSUING WORTHLESS CHECKS', 'AGGRAVATED BATTERY DOMESTIC',
       'EXTORTION (THREATS)', 'AGGRAVATED BATTERY BY SHOOTING',
       'HOMICIDE BY SHOOTING', 'BICYCLE THEFT', 'SIMPLE ROBBERY'

2b. Create a function that generates a new "Crime Category" column using "if/else" structure with incident "Type" categorized accordingly. Markdown notes are included in the cell in order to better keep track of incident groupings.

In [11]:
#Creating categories for each crime type in a new column named 'Crime Category'

def check_condition(row):
    crime = row['Type']
    if crime in ['BUSINESS BURGLARY', 'THEFT', 'PROWLER', 'PICKPOCKET', 'AUTO THEFT', 'SHOPLIFTING', 'SIMPLE BURGLARY VEHICLE', 'UNAUTHORIZED USE OF VEHICLE', 'THEFT FROM EXTERIOR', 'SIMPLE BURGLARY', 'BICYCLE THEFT', 'RESIDENCE BURGLARY', 'THEFT BY FRAUD', 'THEFT BY EMBEZZLEMENT', 'POSSESSION OF STOLEN PROPERTY', 'SAFE BURGLARY', 'SIMPLE ROBBERY', 'SIMULTANEOS STOLEN/RECOVERY VEHICLE', 'THEFT FROM EXTERIOR OF VEHICLE', 'ARMED ROBBERY WITH GUN', 'ARMED ROBBERY WITH KNIFE', 'SIMPLE ROBBERY, PROPERTY SNATCHING', 'AGGRAVATED BURGLARY DOMESTIC', 'ARMED ROBBERY', 'AGGRAVATED BURGLARY', 'RECOVERY OF REPORTED STOLEN VEHICLE', 'SIMPLE BURGLARY DOMESTIC']:
        return 'Theft and Burglary'
# Business Burglary
# Theft
# Prowler
# Pickpocket
# Auto Theft
# Shoplifting
# Simple Burglary Vehicle
# Unauthorized Use of Vehicle
# Theft from Exterior
# Simple Burglary
# Bicycle Theft
# Residence Burglary
# Theft by Fraud
# Theft by Embezzlement
# Possession of Stolen Property
# Safe Burglary
# Simple Robbery
# Simultaneous Stolen/Recovery Vehicle
# Theft from Exterior of Vehicle
# Armed Robbery with Gun
# Armed Robbery with Knife
# Simple Robbery, Property Snatching
# Aggravated Burglary Domestic
# Armed Robbery
# Aggravated Burglary
# Recovery of Reported Stolen Vehicle
# Simple Burglary Domestic
    
    elif crime in ['FIGHT', 'AGGRAVATED ASSAULT', 'SIMPLE BATTERY', 'SIMPLE BATTERY DOMESTIC', 'AGGRAVATED BATTERY DOMESTIC', 'AGGRAVATED BATTERY BY SHOOTING', 'AGGRAVATED BATTERY', 'AGGRAVATED ASSAULT DOMESTIC', 'AGGRAGATED BATTERY BY KNIFE', 'AGGRAVATED BATTERY DOMESTIC', 'AGGRAVATED CRIMINAL DAMAGE', 'HIT & RUN WITH INJURIES','SIMPLE ASSAULT','SIMPLE ASSAULT DOMESTIC', 'CARJACKING', 'CARJACKING- NO WEAPON', 'SIMPLE ARSON', 'AGGRAVATED ARSON', 'ARMED ROBBERY', 'AGGRAVATED KIDNAPPING', 'SIMPLE ARSON DOMESTIC', 'AGGRAVATED BATTERY BY CUTTING', 'HOMICIDE BY SHOOTING', 'HOMICIDE BY CUTTING', 'HOMICIDE DOMESTIC', 'HOMICIDE', 'AGGRAVATED BATTERY BY KNIFE', ]:
        return 'Violent Crimes'


# Fight
# Aggravated Assault
# Simple Battery
# Simple Battery Domestic
# Aggravated Battery Domestic
# Aggravated Battery by Shooting
# Aggravated Battery
# Aggravated Assault Domestic
# Aggravated Battery by Knife
# Aggravated Battery Domestic
# Aggravated Criminal Damage
# Hit & Run with Injuries
# Simple Assault
# Simple Assault Domestic
# Carjacking
# Carjacking- No Weapon
# Simple Arson
# Aggravated Arson
# Aggravated Kidnapping
# Simple Arson Domestic
# Aggravated Battery by Cutting
# Homicide by Shooting
# Homicide by Cutting
# Homicide Domestic
# Homicide
# Aggravated Battery by Knife

    elif crime in ['DRUNK', 'DRUG VIOLATIONS', 'DRIVING WHILE UNDER INFLUENCE', 'PUBLIC INTOXICATION', 'UNDERAGE DRINKING VIOLATION', 'CONTRIBUTING TO DELINQUENCY']:
        return 'Substance Related / Disorderly Conduct'
# Drunk
# Drug Violations
# Driving While Under Influence
# Public Intoxication
# Underage Drinking Violation
# CONTRIBUTING TO DELINQUENCY	

    elif crime in ['SIMPLE CRIMINAL DAMAGE', 'CRIMINAL MISCHIEF', 'CRIMINAL DAMAGE DOMESTIC', 'BLIGHTED PROPERTY', 'DESECRATION OF GRAVES', 'CRIMINAL MISCHIEF DOMESTIC']:
        return 'Property Damage & Vandalism'
# Simple Criminal Damage
# Criminal Mischief
# Criminal Damage Domestic
# Aggravated Criminal Damage
# Blighted Property
# Desecration of Graves
# Criminal Mischief Domestic
    elif crime in ['INDECENT BEHAVIOR', 'SEX OFFENDER REGISTRATION CHECK', 'VIDEO VOYEURISM', 'OBSCENITY, EXPOSING', 'PEEPING TOM']:
        return 'Sexual Offenses & Related Crimes'
# Indecent Behavior
# Sex Offender Registration Check
# Video Voyeurism
# Obscenity, Exposing
# Peeping Tom
    elif crime in ['DEATH', 'UNCLASSIFIED DEATH', 'SUSPICIOUS PERSON', 'DOMESTIC DISTURBANCE', 'BURGLAR ALARM, SILENT', 'BURGLAR ALARM, LOCAL', 'NOISE COMPLAINT', 'FIREWORKS', 'TRUANT VIOLATION', 'QUALITY OF LIFE ISSUE', 'ISSUING WORTHLESS CHECKS', 'EXTORTION (THREATS)', 'FUGITIVE ATTACHMENT', 'HOMELESS', 'HOLD UP ALARM', 'UNATTENDED PACKAGE', 'PROTEST', 'PANDERING', 'GAMBLING', 'ASSET SEIZURE', 'BURGLAR ALARM, SILENT, FAR NO RESPONSE', 'CURFEW VIOLATION', 'VIOLATION OF PROTECTION ORDER']:
        return 'Misc.'
# Death
# Unclassified Death
# Suspicious Person
# Domestic Disturbance
# Noise Complaint
# Fireworks
# Truant Violation
# Quality of Life Issue
# Issuing Worthless Checks
# Extortion (Threats)
# Fugitive Attachment
# Homeless
# Hold Up Alarm
# Unattended Package
# Protest
# Pandering
# Gambling
# Asset Seizure

# Curfew Violation
# Violation of Protection Order

    elif crime in ['MISSING ADULT', 'RECKLESS DRIVING', 'FIRE', 'DISCHARGING FIREARM', 'BOMB SCARE', 'HIT & RUN', 'ILLEGAL CARRYING OF WEAPON', 'ILLEGAL CARRYING OF WEAPON- GUN', 'ILLEGAL CARRYING OF WEAPON- KNIFE', 'OFFICER NEEDS ASSISTANCE, LIFE IN DANGER', 'NEGLIENT INJURY', 'SIMPLE KIDNAPPING', 'SUSPICIOUS PACKAGE', 'HIT & RUN POLICE VEHICLE', 'IMPERSONATING AN OFFICER', 'HIT & RUN FATALITY', 'EXPLOSION', 'FORGERY', 'EXTORTION (THREATS) DOMESTIC', 'CRUELTY TO ANIMALS', 'HIT & RUN CITY VEHICLE', 'VEHICLE PURSUIT', 'DISPERSE SUBJECTS', 'AGGRAVATED ESCAPE', 'HOSTAGE SITUATION', 'VEHICLE PURSUIT', 'SIMPLE ESCAPE', 'AGGRAVATED ESCAPE']:
        return 'Severe Threats & Public Safety Concerns'
# Missing Adult
# Reckless Driving
# Fire
# Discharging Firearm
# Bomb Scare
# Hit & Run
# Illegal Carrying of Weapon
# Illegal Carrying of Weapon- Gun
# Illegal Carrying of Weapon- Knife
# Officer Needs Assistance, Life in Danger
# Negligent Injury - ***SPELLED WRONG as "NEGLIENT"
# Simple Kidnapping
# Suspicious Package
# Hit & Run Police Vehicle
# Impersonating an Officer
# Hit & Run Fatality
# Explosion
# Forgery
# Extortion (Threats) Domestic
# Cruelty to Animals   
# Hit & Run City Vehicle
# Vehicle Pursuit
# Disperse Subjects
# Aggravated Escape
# Hostage Situation
# Vehicle Pursuit
# Simple Escape
# Aggravated Escape
    

    else:
        return 'Other'
        

# Apply the function to create a new column
result['Crime Category'] = result.apply(check_condition, axis=1)

In [12]:
result.head(50)

,Code,Type,Time,Beat,Address,Zip,District,Neighborhood,Latitude,Longitude,Crime Category
0,107,SUSPICIOUS PERSON,2018-02-08 16:55:15,2K03,018XX Cambronne St,70118.0,2,LEONIDAS,29.954301,-90.127416,Misc.
1,107,SUSPICIOUS PERSON,2018-02-08 17:20:45,5I04,Andry St & N Claiborne Ave,70117.0,5,LOWER NINTH WARD,29.966844,-90.016829,Misc.
2,62B,BUSINESS BURGLARY,2018-02-08 05:19:41,7L01,046XX Michoud Blvd,70129.0,7,VILLAGE DE LEST,30.032034,-89.928708,Theft and Burglary
3,29,DEATH,2018-02-08 10:16:26,6P01,014XX General Taylor St,70115.0,6,TOURO,29.925929,-90.096461,Misc.
4,67,THEFT,2018-02-08 10:54:10,5L01,023XX N Tonti St,70117.0,5,ST. ROCH,29.980707,-90.054597,Theft and Burglary
5,62A,"BURGLAR ALARM, SILENT",2018-02-08 06:37:28,2S01,033XX Hamilton St,70118.0,2,HOLLYGROVE,29.969441,-90.121415,Misc.
6,63,PROWLER,2018-02-10 02:15:36,8D05,005XX Bourbon St,70130.0,8,FRENCH QUARTER,29.957462,-90.066532,Theft and Burglary
7,103D,DOMESTIC DISTURBANCE,2018-02-08 21:47:46,5G03,026XX N Galvez St,70117.0,5,ST. CLAUDE,29.978933,-90.046503,Misc.
8,103F,FIGHT,2018-02-08 21:57:29,6D02,Conery St & Saint Charles Ave,70115.0,6,CENTRAL CITY,29.930490,-90.086623,Violent Crimes
9,94,DISCHARGING FIREARM,2018-02-08 22:57:37,3R03,Prentiss Ave & Saint Roch Ave,70122.0,3,MILNEBURG,30.017114,-90.055842,Severe Threats & Public Safety Concerns


### 3. Remove burglar alarm row records as they do not serve as a reliable incident type (eg. - system malfunctions lead to incident count outliers).

3a. Use the the bitwise NOT operation to create a dataframe that excludes "Burglar Alarm, Silent", "Burglar Alarm, Local", and "Burglar alarm, silent, far no response" row records. 

In [15]:
# Dropping Burglar Alarm Values
result = result[~((result['Type'] == 'BURGLAR ALARM, SILENT') | (result['Type'] == 'BURGLAR ALARM, LOCAL') | (result['Type'] == 'BURGLAR ALARM, SILENT, FAR NO RESPONSE'))]
# Burglar Alarm, Silent
# Burglar Alarm, Local
# BURGLAR ALARM, SILENT, FAR NO RESPONSE
                                                                                                              

In [16]:
result.head(30)

,Code,Type,Time,Beat,Address,Zip,District,Neighborhood,Latitude,Longitude,Crime Category,Date,Time of Day
0,107,SUSPICIOUS PERSON,2018-02-08 16:55:15,2K03,018XX Cambronne St,70118.0,2,LEONIDAS,29.954301,-90.127416,Misc.,2018-02-08,16:55:15
1,107,SUSPICIOUS PERSON,2018-02-08 17:20:45,5I04,Andry St & N Claiborne Ave,70117.0,5,LOWER NINTH WARD,29.966844,-90.016829,Misc.,2018-02-08,17:20:45
2,62B,BUSINESS BURGLARY,2018-02-08 05:19:41,7L01,046XX Michoud Blvd,70129.0,7,VILLAGE DE LEST,30.032034,-89.928708,Theft and Burglary,2018-02-08,05:19:41
3,29,DEATH,2018-02-08 10:16:26,6P01,014XX General Taylor St,70115.0,6,TOURO,29.925929,-90.096461,Misc.,2018-02-08,10:16:26
4,67,THEFT,2018-02-08 10:54:10,5L01,023XX N Tonti St,70117.0,5,ST. ROCH,29.980707,-90.054597,Theft and Burglary,2018-02-08,10:54:10
6,63,PROWLER,2018-02-10 02:15:36,8D05,005XX Bourbon St,70130.0,8,FRENCH QUARTER,29.957462,-90.066532,Theft and Burglary,2018-02-10,02:15:36
7,103D,DOMESTIC DISTURBANCE,2018-02-08 21:47:46,5G03,026XX N Galvez St,70117.0,5,ST. CLAUDE,29.978933,-90.046503,Misc.,2018-02-08,21:47:46
8,103F,FIGHT,2018-02-08 21:57:29,6D02,Conery St & Saint Charles Ave,70115.0,6,CENTRAL CITY,29.930490,-90.086623,Violent Crimes,2018-02-08,21:57:29
9,94,DISCHARGING FIREARM,2018-02-08 22:57:37,3R03,Prentiss Ave & Saint Roch Ave,70122.0,3,MILNEBURG,30.017114,-90.055842,Severe Threats & Public Safety Concerns,2018-02-08,22:57:37
10,35D,SIMPLE BATTERY DOMESTIC,2018-02-08 23:44:48,4C02,033XX Garden Oaks Dr,70114.0,4,TALL TIMBERS - BRECHTEL,29.922545,-90.022647,Violent Crimes,2018-02-08,23:44:48


### Export resulting dataframe as a .csv file to be loaded as a data source in the Tableau project. 

In [19]:
result.to_csv('nolacrimecleanest.csv', index=False)